<a href="https://colab.research.google.com/github/elylaila/seminario/blob/main/Explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gsarti/lcl23-xnlm-lab/blob/main/notebooks/2_Attribution_Contrastive.ipynb)

In [1]:
%%capture
# Run in Colab to install local packages
%pip install ferret-xai bitsandbytes accelerate numpy==1.23.1
%pip install git+https://github.com/huggingface/transformers.git
# %pip install git+https://github.com/inseq-team/inseq.git

In [116]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import pandas as pd
from ferret import Benchmark
import torch
import numpy as np
from tqdm.notebook import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Seminario/

Mounted at /content/drive
/content/drive/MyDrive/Seminario


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [15]:
name = 'dbmdz/bert-base-italian-cased' # inserire bert-ita
model = AutoModelForSequenceClassification.from_pretrained('final_model/checkpoint-4375').to(device) # inserire path modello finale
tokenizer = AutoTokenizer.from_pretrained(name)
print(model.config.id2label)

{0: 'LABEL_0', 1: 'LABEL_1'}


In [5]:
predictions = pd.read_csv('final_model_predictions.csv', usecols=['y_pred', 'y_true'])
predictions

,y_true,y_pred
0,1,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
995,0,0
996,0,1
997,0,0
998,1,1


In [55]:
df = pd.read_csv('dataset/HODI_2023_test_GOLD.tsv', sep='\t').drop(columns=['homotransphobic'])
df = pd.concat([df, predictions], axis=1)
df

,id,text,rationales,y_true,y_pred
0,5001,@user_abcdefg @user_abc Quasi quasi è meglio f...,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",1,0
1,5002,"@user_abcdefgh Sotto quel tweet, c'è un ammass...",NaN,0,0
2,5003,@user_a brava Mara come sempre puntatona.. Sei...,NaN,0,0
3,5004,copio i gay cagatemi !!! se vi chiedessi di de...,NaN,0,0
4,5005,io: è bono eh papà: ma non eri frocia te 🤡🤡🤡,NaN,0,0
...,...,...,...,...,...
995,5996,RT @user_abcdefghij @user_abcdefghij Quindi tu...,NaN,0,0
996,5997,@user_abcde Quel “sono a lavoro André” che suo...,NaN,0,1
997,5998,@user_abcdefghij @user_a 115 gol e 48 assist i...,NaN,0,0
998,5999,@user_abcdefgh Ma non rompermi le balle culatt...,"[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 5...",1,1


In [149]:
random_state = 42

# List of conditions
conditions = [
    {'y_true_val': 0, 'y_pred_val': 1, 'drop_columns': ['rationales'], 'target': 1},
    {'y_true_val': 1, 'y_pred_val': 1, 'drop_columns': [], 'target': 1},
    {'y_true_val': 1, 'y_pred_val': 0, 'drop_columns': ['rationales'], 'target': 0}
]

# Define a function to extract rationales
def get_rationales(rationales_str, frase):
    # Convert string of numbers to a list of integers
    rationales_lst = [int(e) for e in rationales_str[1:-1].split(', ')]

    # Use list comprehension to get characters based on indices
    rationales_ch = ''.join(frase[ch_idx] for ch_idx in rationales_lst)

    return rationales_ch

# Store explanations and rationales for each condition
explanations_dict = {}
rationales_dicts = {}
dfs_samples_dict = {}

for condition in tqdm(conditions):
    filtered_df = df[(df['y_true'] == condition['y_true_val']) & (df['y_pred'] == condition['y_pred_val'])]

    if condition['drop_columns']:
        filtered_df = filtered_df.drop(columns=condition['drop_columns'])


    sampled_df = filtered_df.sample(n=5, random_state=random_state)
    sentences = list(sampled_df['text'])
    df_name = f"df_errors_{condition['y_true_val']}{condition['y_pred_val']}_sample"
    dfs_samples_dict[df_name] = sampled_df

    explanations = [Benchmark(model, tokenizer).explain(sentence, target=condition['target']) for sentence in tqdm(sentences)]

    explanations_dict[f"explanations_{condition['y_true_val']}{condition['y_pred_val']}"] = explanations

    if 'rationales' in sampled_df:
      rationales_strs = list(sampled_df['rationales'])
      rationales = [get_rationales(rationales_str, sentence) for rationales_str, sentence in zip(rationales_strs, sentences)]
      rationales_dict = dict(zip(sentences, rationales))
      rationales_dicts[f"rationales_{condition['y_true_val']}{condition['y_pred_val']}"] = rationales_dict



  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

...

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

In [136]:
i = 0
exp = explanations_dict['explanations_01'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcdef @user_abcdef 4) se un ragazzo desse un pugno in faccia ad una ragazza perché ha insultato uno con un "checca di merda" avrebbe non solo il mio appoggio ma anche la mia ammirazione.


,@,us,##er,_,ab,##c,##de,##f,@.1,us.1,##er.1,_.1,ab.1,##c.1,##de.1,##f.1,4,),se,un,ragazzo,des,##se,un.1,pugno,in,faccia,ad,una,ragazza,perché,ha,insul,##tato,uno,con,un.2,"""",che,##cca,di,merda,""".1",avrebbe,non,solo,il,mio,appoggio,ma,anche,la,mia,ammira,##zione,.
Partition SHAP,0.02,-0.10,-0.02,-0.01,-0.01,-0.00,-0.00,-0.02,0.05,-0.03,-0.00,-0.06,-0.03,0.01,0.02,-0.01,0.02,0.01,0.00,-0.02,-0.01,-0.02,-0.04,-0.01,-0.02,-0.02,-0.02,-0.02,-0.03,-0.02,-0.02,-0.01,-0.00,-0.00,-0.00,-0.00,0.04,0.04,0.07,0.07,0.00,0.00,-0.00,-0.00,0.01,0.01,0.01,0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.02,0.00
LIME,0.00,-0.01,-0.01,0.00,0.00,-0.00,-0.01,0.01,0.01,-0.01,0.00,0.00,0.00,-0.01,0.01,-0.01,0.01,0.00,-0.00,-0.02,-0.02,-0.03,-0.00,-0.00,-0.04,-0.02,0.00,0.01,-0.01,-0.07,0.00,0.00,-0.04,-0.01,0.00,-0.00,0.08,-0.00,0.16,0.05,0.04,0.07,-0.16,0.01,0.01,0.00,0.01,-0.00,-0.00,-0.01,-0.00,0.01,0.00,-0.00,0.00,-0.00
Gradient,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.03,0.01,0.02,0.01,0.01,0.02,0.03,0.01,0.07,0.02,0.02,0.03,0.03,0.06,0.05,0.05,0.02,0.06,0.03,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.03
Gradient (x Input),-0.02,-0.01,0.00,0.01,-0.00,0.00,0.00,0.01,-0.01,-0.01,-0.00,0.01,0.01,0.00,-0.00,0.00,0.02,-0.03,0.00,-0.00,0.00,-0.00,-0.01,0.01,-0.01,-0.03,-0.04,-0.01,-0.01,0.03,0.06,-0.03,-0.06,-0.02,-0.01,0.05,0.01,-0.06,0.08,0.01,-0.04,-0.10,0.01,-0.01,0.01,0.01,-0.00,-0.01,0.02,0.01,0.00,-0.00,0.00,0.02,0.00,-0.04
Integrated Gradient,0.00,0.01,0.00,-0.01,0.01,0.00,0.01,0.00,0.03,0.02,0.00,0.01,-0.00,0.00,0.01,-0.00,-0.02,0.01,0.01,0.00,0.04,-0.03,0.01,-0.03,-0.02,0.01,-0.01,-0.01,-0.02,0.01,0.01,0.01,0.10,0.02,0.04,0.01,0.05,-0.07,-0.04,-0.05,0.05,0.03,0.03,0.04,0.00,0.02,0.02,0.00,-0.01,-0.00,0.00,0.00,0.02,0.00,-0.00,0.00
Integrated Gradient (x Input),-0.01,0.00,-0.00,0.01,0.00,-0.00,0.01,-0.00,-0.01,0.00,-0.00,0.01,0.00,0.00,-0.00,-0.00,-0.01,-0.03,-0.02,-0.02,0.00,-0.02,-0.01,0.02,-0.04,-0.02,0.00,-0.00,-0.02,0.00,-0.02,-0.03,-0.01,-0.01,-0.00,0.06,0.06,-0.08,0.11,0.06,-0.02,0.12,-0.05,-0.01,0.01,0.00,0.01,-0.01,0.00,0.00,-0.01,0.00,0.00,0.02,0.00,-0.00


In [89]:
i = 1
exp = explanations_dict['explanations_01'][i]
print(exp[i].text)
bench.show_table(exp)

comunque siamo 5 froci in una piccola macchi a diretti verso una serata frocia


,comunque,siamo,5,fro,##ci,in,una,piccola,macch,##i,a,diretti,verso,una.1,serata,fro.1,##cia
Partition SHAP,-0.02,0.06,0.09,0.26,0.01,0.01,-0.05,-0.10,0.03,-0.03,-0.01,0.02,0.02,-0.03,-0.13,-0.06,-0.07
LIME,0.01,0.04,0.06,0.42,0.02,0.05,-0.05,-0.06,-0.00,-0.05,0.04,0.04,-0.02,-0.02,-0.06,0.01,-0.04
Gradient,0.05,0.05,0.06,0.14,0.07,0.03,0.03,0.04,0.08,0.03,0.02,0.05,0.03,0.03,0.08,0.04,0.07
Gradient (x Input),-0.15,-0.03,-0.07,-0.15,0.03,-0.05,0.00,0.00,-0.08,-0.03,-0.04,0.07,-0.01,-0.01,0.03,0.03,-0.13
Integrated Gradient,-0.01,-0.00,-0.04,-0.27,-0.02,-0.03,0.12,0.02,-0.08,0.01,-0.01,-0.05,-0.03,0.03,-0.12,0.01,-0.09
Integrated Gradient (x Input),0.06,0.03,-0.06,0.23,0.11,-0.07,-0.08,-0.03,0.02,-0.01,-0.02,-0.00,0.02,-0.05,-0.11,-0.09,-0.02


In [90]:
i = 2
exp = explanations_dict['explanations_01'][i]
print(exp[i].text)
bench.show_table(exp)

AUG 19 2022, 19:36: BERNARDINO FEMMINIELLI [URL]


,A,##UG,19,202,##2,",",19.1,:,36,:.1,BE,##R,##NA,##R.1,##DI,##NO,FE,##M,##MIN,##IE,##LL,##I,[,UR,##L,]
Partition SHAP,0.01,0.01,0.00,0.01,-0.00,0.01,-0.03,-0.07,0.00,0.04,-0.06,0.01,-0.09,-0.02,-0.05,-0.09,0.01,-0.02,-0.04,0.00,-0.04,0.04,-0.09,0.02,-0.18,-0.06
LIME,0.03,0.01,-0.02,-0.01,-0.05,-0.01,-0.01,-0.00,-0.01,-0.03,-0.09,0.00,-0.02,-0.03,0.03,-0.07,0.01,-0.07,-0.01,-0.01,-0.06,-0.03,-0.13,0.05,-0.12,-0.09
Gradient,0.02,0.04,0.02,0.05,0.02,0.03,0.03,0.03,0.03,0.04,0.04,0.02,0.03,0.04,0.08,0.06,0.05,0.04,0.03,0.03,0.04,0.03,0.03,0.06,0.03,0.03
Gradient (x Input),0.04,-0.02,0.07,-0.03,0.01,-0.02,0.03,-0.01,0.01,0.04,0.03,0.01,-0.04,-0.04,0.10,-0.06,0.00,0.12,0.07,0.01,0.08,0.01,0.02,0.02,0.04,0.04
Integrated Gradient,0.05,-0.03,0.03,-0.07,0.03,-0.02,0.01,-0.05,0.00,-0.06,-0.02,-0.03,0.03,0.03,-0.14,0.01,-0.04,-0.00,0.08,0.07,0.03,0.02,0.02,0.04,-0.01,0.01
Integrated Gradient (x Input),-0.03,0.06,0.01,0.01,-0.02,0.02,-0.05,-0.07,0.05,-0.02,0.03,-0.05,0.01,-0.06,0.06,-0.19,0.01,-0.08,-0.04,-0.03,0.02,-0.00,-0.03,-0.01,-0.04,0.00


In [91]:
i = 3
exp = explanations_dict['explanations_01'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcdefg @user_abcdefgh @user_abcde Io preferisco fidarmi da chi non nasconde il suo volto ma come sempre la gente che pensa di fare il “culattone” col le terga dell’altro abbonda ¯\_(ツ)_/¯


,@,us,##er,_,ab,##c,##de,##f,##g,@.1,us.1,##er.1,_.1,ab.1,##c.1,##de.1,##f.1,##gh,@.2,us.2,##er.2,_.2,ab.2,##c.2,##de.2,Io,preferisco,fidarmi,da,chi,non,nasconde,il,suo,volto,ma,come,sempre,la,gente,che,pensa,di,fare,il.1,“,cul,##atto,##ne,”,col,le,ter,##ga,dell,’,altro,abbond,##a,[UNK],\,_.3,(,[UNK].1,),_.4,/,[UNK].2
Partition SHAP,0.01,-0.02,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.01,-0.00,-0.01,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.01,-0.00,-0.01,0.00,-0.00,-0.00,-0.00,-0.01,-0.01,-0.00,-0.01,-0.02,-0.00,0.01,-0.01,-0.00,0.00,-0.01,-0.00,-0.00,-0.01,-0.00,-0.00,0.01,0.04,0.10,-0.07,0.11,0.14,0.05,0.00,0.02,0.01,0.02,-0.02,0.02,0.00,0.09,0.01,-0.03,-0.00,-0.00,-0.02,0.00,0.00,0.00,-0.02,-0.01,-0.00
LIME,0.01,-0.00,-0.01,-0.00,-0.00,-0.01,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.01,-0.01,0.00,0.00,0.00,0.01,0.00,-0.00,0.00,0.01,-0.01,-0.00,-0.00,0.01,0.01,0.00,-0.01,0.01,-0.00,-0.00,0.02,-0.00,-0.03,0.00,0.00,-0.01,-0.01,-0.05,-0.02,0.01,0.02,0.02,0.06,0.01,0.13,0.16,0.05,-0.01,0.06,-0.01,0.01,-0.02,0.01,0.00,0.03,0.02,-0.03,-0.02,-0.01,0.00,-0.00,0.01,-0.01,-0.00,-0.00,0.01
Gradient,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.04,0.01,0.02,0.02,0.02,0.02,0.02,0.03,0.06,0.02,0.02,0.02,0.02,0.05,0.03,0.02,0.01,0.02,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
Gradient (x Input),0.03,-0.00,-0.01,0.01,0.00,-0.01,-0.01,0.00,-0.01,0.01,-0.01,-0.01,0.00,0.00,-0.01,-0.00,0.00,-0.00,0.01,-0.00,-0.01,-0.00,-0.00,-0.01,-0.01,-0.05,-0.00,-0.03,0.01,0.01,-0.00,0.03,-0.01,0.00,0.00,0.01,0.01,-0.01,-0.00,0.03,-0.02,0.00,-0.02,0.03,0.00,-0.01,0.00,0.05,0.05,0.05,0.01,-0.01,0.05,0.04,-0.04,0.02,0.04,0.04,0.02,0.00,-0.00,-0.00,-0.02,0.01,-0.03,-0.00,-0.01,0.01
Integrated Gradient,0.01,-0.00,0.00,-0.01,0.00,0.00,-0.00,-0.01,0.00,-0.00,-0.00,-0.00,0.01,0.00,-0.00,0.02,0.01,0.04,0.01,0.00,-0.01,0.01,0.01,-0.01,-0.00,0.02,0.01,-0.01,-0.00,0.01,-0.01,0.00,0.01,-0.01,-0.00,-0.00,0.01,-0.00,0.01,0.01,0.01,-0.07,-0.02,0.03,-0.04,0.00,-0.02,-0.02,0.01,-0.00,-0.00,0.00,0.00,0.04,-0.07,-0.05,-0.05,0.06,0.00,-0.01,0.01,-0.01,0.02,-0.00,-0.03,0.01,-0.01,0.01
Integrated Gradient (x Input),0.00,-0.00,-0.01,0.01,-0.01,-0.01,-0.02,-0.00,-0.01,0.01,-0.01,-0.02,0.01,-0.02,0.00,-0.01,0.01,-0.04,0.02,-0.00,-0.02,0.00,-0.01,0.00,-0.02,-0.01,0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.00,-0.02,0.02,-0.01,-0.02,-0.01,-0.05,-0.06,-0.04,-0.01,-0.03,0.06,-0.01,0.02,0.08,0.02,0.02,0.01,-0.00,0.00,-0.01,-0.06,-0.02,0.01,-0.00,-0.03,-0.00,-0.01,-0.00,0.02,-0.01,-0.02,-0.00,0.00,0.00


In [92]:
i = 4
exp = explanations_dict['explanations_01'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcd Pensa , prima di diceva ricchione...


,@,us,##er,_,ab,##c,##d,Pensa,",",prima,di,diceva,ricchi,##one,.,..1,..2
Partition SHAP,-0.03,-0.04,-0.02,0.01,-0.03,-0.01,-0.02,0.06,0.16,-0.12,0.03,-0.22,0.12,0.03,0.01,0.08,0.00
LIME,-0.01,-0.02,-0.00,0.00,-0.03,-0.02,-0.02,0.00,0.07,-0.11,0.11,-0.20,0.24,0.09,0.06,0.02,0.01
Gradient,0.05,0.03,0.02,0.03,0.03,0.03,0.02,0.06,0.04,0.06,0.08,0.17,0.15,0.07,0.03,0.02,0.02
Gradient (x Input),0.03,-0.01,-0.03,0.00,0.08,-0.03,-0.04,-0.00,0.01,0.10,-0.02,0.09,-0.35,0.03,0.04,0.03,0.05
Integrated Gradient,0.15,-0.03,0.01,-0.00,0.06,0.01,0.03,0.06,-0.08,-0.16,0.13,0.04,0.02,0.02,-0.01,-0.05,-0.08
Integrated Gradient (x Input),0.01,-0.02,-0.02,-0.02,0.04,-0.03,-0.02,-0.04,0.12,-0.06,-0.00,-0.27,-0.21,0.09,0.03,-0.01,0.02


In [94]:
i = 0
exp = explanations_dict['explanations_11'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcdefghi ma sei rotto in culo tu che cazzo significa


Token,@,us,##er,_,ab,##c,##de,##f,##ghi,ma,sei,rotto,in,culo,tu,che,cazzo,significa
Partition SHAP,0.10,-0.05,-0.05,-0.03,0.02,-0.02,-0.01,-0.00,0.04,0.08,0.03,0.10,0.00,0.12,-0.07,-0.06,0.01,-0.21
LIME,0.05,-0.04,-0.01,-0.01,-0.02,-0.00,0.03,0.03,0.03,0.04,0.01,0.26,0.09,0.18,-0.13,-0.02,0.00,-0.04
Gradient,0.05,0.03,0.02,0.03,0.02,0.03,0.02,0.03,0.04,0.05,0.06,0.10,0.04,0.08,0.08,0.04,0.05,0.10
Gradient (x Input),-0.01,-0.02,-0.01,0.02,-0.03,-0.04,-0.03,0.02,-0.03,0.02,0.02,0.03,-0.04,-0.12,-0.19,-0.10,-0.02,-0.04
Integrated Gradient,-0.04,-0.01,-0.05,0.04,0.01,0.07,0.00,0.00,-0.06,0.02,0.03,0.26,0.01,0.05,-0.03,0.07,0.06,0.12
Integrated Gradient (x Input),-0.02,-0.00,-0.00,0.00,-0.01,-0.01,-0.01,0.02,0.03,0.01,0.17,-0.02,-0.01,0.20,-0.21,-0.04,-0.00,-0.23


In [96]:
i = 1
exp = explanations_dict['explanations_11'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcdef Certo è un noto culattone @user_abcdefghij la carriera l‘ha fatta con Arpa testa bassa e culo alzato


,@,us,##er,_,ab,##c,##de,##f,Certo,è,un,noto,cul,##atto,##ne,@.1,us.1,##er.1,_.1,ab.1,##c.1,##de.1,##f.1,##ghi,##j,la,carriera,l,‘,ha,fatta,con,Ar,##pa,testa,bassa,e,culo,alzato
Partition SHAP,0.02,-0.01,-0.00,-0.03,-0.01,0.01,-0.00,0.03,0.01,0.02,0.09,0.09,-0.02,0.04,-0.01,-0.01,-0.02,-0.01,-0.03,-0.04,-0.02,0.01,0.03,0.04,0.00,0.01,-0.04,0.01,-0.00,-0.03,-0.07,-0.03,-0.03,0.04,0.04,-0.04,0.01,-0.04,-0.00
LIME,0.02,-0.01,-0.01,-0.02,-0.02,0.01,0.02,0.02,-0.01,0.05,0.10,0.09,-0.00,0.09,-0.00,-0.04,-0.02,-0.02,-0.03,-0.04,-0.02,0.00,0.03,0.05,-0.00,0.01,-0.03,0.01,0.00,0.00,-0.06,-0.04,-0.01,-0.00,0.04,-0.02,-0.00,-0.02,-0.01
Gradient,0.03,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.03,0.03,0.07,0.04,0.07,0.03,0.05,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.02,0.03,0.01,0.06,0.02,0.03,0.01,0.02,0.02,0.01,0.01,0.03,0.02,0.01,0.03,0.02
Gradient (x Input),0.07,-0.02,-0.02,0.01,0.01,-0.00,-0.02,0.00,0.02,-0.03,0.07,0.12,-0.06,-0.00,-0.00,-0.02,-0.03,-0.04,-0.01,0.04,-0.00,0.00,-0.00,-0.01,-0.03,0.01,0.04,0.00,0.02,-0.00,0.01,0.00,0.01,0.00,-0.05,-0.01,-0.01,-0.01,0.01
Integrated Gradient,-0.05,0.04,-0.02,-0.01,-0.00,-0.02,0.01,-0.00,-0.01,-0.06,0.04,-0.15,0.01,-0.04,-0.01,-0.05,-0.03,0.01,0.00,-0.02,-0.00,-0.00,-0.03,0.02,0.00,-0.02,-0.04,0.00,-0.05,-0.03,-0.00,-0.05,0.03,0.01,-0.02,0.01,-0.01,-0.03,-0.03
Integrated Gradient (x Input),0.04,0.00,-0.04,0.00,-0.02,-0.01,-0.03,-0.03,0.09,-0.03,0.01,0.06,-0.10,0.07,0.05,0.03,0.00,-0.04,-0.01,-0.00,-0.00,0.01,-0.03,0.02,-0.02,-0.01,-0.03,-0.01,0.03,-0.03,-0.03,0.01,-0.02,-0.00,-0.02,-0.03,0.01,-0.03,-0.00


In [97]:
i = 2
exp = explanations_dict['explanations_11'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcd CHI ? AHHH QUESTO COGLIONE RICCHIONE SODOMITA PEDOFILO… 😂… HA ANCHE LA TESTA A FORMA DI GLANDE. STAI ATTENTO PELATO, CHE SIAMO IN TANTI A PISCIARTI IN TESTA, TI RICRESCONO I CAPELLI POI… NON ABBOCCATE SONO BALLE. IL PRIMO SCHIAFFO LO PRENDE LUI. 🎪🤡🤡🤡


,@,us,##er,_,ab,##c,##d,CHI,?,A,##H,##H.1,##H.2,QU,##ESTO,CO,##GL,##IONE,RIC,##CHI,##ONE,SO,##DO,##MI,##TA,PE,##DO.1,##FI,##LO,[UNK],[UNK].1,[UNK].2,HA,AN,##CHE,LA,TE,##ST,##A,A.1,FORMA,DI,G,##LAN,##DE,.,STA,##I,AT,##TEN,##TO,PE.1,##LA,##TO.1,",",CHE,SI,##AM,##O,IN,T,##ANTI,A.2,PI,##SC,##IA,##RT,##I.1,IN.1,TE.1,##ST.1,##A.1,",.1",TI,RIC.1,##RE,##SC.1,##ONO,I,CAP,##ELL,##I.2,PO,##I.3,[UNK].3,NON,ABB,##OC,##CA,##TE,SONO,B,##ALL,##E,..1,IL,PR,##IMO,SCH,##IA.1,##FF,##O.1,LO,PRE,##N,##DE.1,LU,##I.4,..2,[UNK].4
Partition SHAP,0.02,-0.02,-0.01,-0.04,-0.06,0.00,-0.01,-0.01,-0.08,-0.00,-0.00,-0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.01,-0.01,-0.03,-0.01,-0.01,-0.01,-0.00,-0.00,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,-0.06,-0.00,-0.00,-0.01,-0.02,-0.00,0.01,0.00,0.00,-0.00,-0.00,-0.05,-0.03,-0.01,-0.02,-0.01,-0.00,-0.00,-0.00,-0.00,-0.02,-0.04,-0.03,-0.01,-0.01,0.00,0.01,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.01,-0.01,-0.01,-0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.01,-0.01,-0.00,-0.00,-0.00,-0.01,-0.03,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.02,-0.02,-0.02,-0.02,-0.01,-0.01,-0.00,-0.01
LIME,-0.00,-0.02,-0.00,0.00,-0.01,-0.00,0.01,0.00,-0.02,0.01,-0.02,0.00,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.03,0.01,-0.02,0.00,-0.01,0.01,-0.04,0.02,0.01,0.00,0.01,0.01,-0.01,0.01,-0.00,-0.00,0.00,0.01,-0.01,-0.01,0.02,0.01,0.01,0.00,-0.01,-0.01,-0.03,0.01,0.01,-0.02,-0.01,-0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.04,0.00,0.00,0.00,0.01,0.01,-0.00,-0.00,-0.02,-0.02,-0.00,0.01,-0.00,-0.00,-0.01,-0.01,-0.01,0.02,-0.00,-0.02,-0.00,0.02,0.02,-0.01,0.00,-0.02,-0.01,-0.01,-0.01,-0.02,-0.00,-0.00,-0.01,-0.02,0.00,0.01,-0.00,0.00,0.01,0.02,-0.00,0.01,0.00,0.01,0.01,-0.01,-0.03,0.00,0.00,-0.01,-0.02,-0.00,-0.01
Gradient,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.01,0.02,0.01,0.01,0.02,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.00,0.00,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.01,0.02,0.01,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.01,0.00,0.01,0.01,0.00,0.01,0.00,0.01,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.01
Gradient (x Input),0.01,-0.00,0.00,0.01,-0.00,0.01,0.01,0.01,0.01,0.01,-0.00,-0.00,-0.01,-0.00,0.03,-0.00,-0.01,-0.03,0.00,-0.01,0.03,-0.01,-0.00,0.01,-0.01,0.02,-0.01,0.00,0.00,-0.02,-0.01,-0.01,-0.00,0.00,0.00,-0.01,-0.01,0.01,0.00,-0.00,-0.00,-0.00,-0.01,-0.01,-0.01,0.01,-0.01,0.00,-0.01,0.00,0.01,-0.00,-0.01,-0.01,-0.00,-0.02,-0.01,0.01,0.00,-0.01,-0.00,0.01,0.00,-0.00,0.02,-0.01,0.05,0.02,-0.00,-0.00,0.01,0.00,-0.00,-0.00,-0.01,0.00,0.01,0.00,-0.00,0.01,0.01,0.00,0.01,0.00,0.00,-0.01,0.03,0.06,0.01,0.00,0.02,-0.01,-0.05,-0.01,0.01,0.01,0.01,-0.00,-0.02,-0.00,0.02,-0.01,0.00,-0.00,-0.01,-0.00,0.00,-0.00,0.02,0.01
Integrated Gradient,-0.01,0.00,0.00,-0.01,0.00,0.00,-0.00,0.00,0.01,-0.00,0.00,-0.01,0.00,-0.01,-0.02,-0.01,-0.02,0.01,-0.01,0.01,-0.04,0.04,0.00,-0.00,0.03,-0.01,-0.01,-0.02,-0.00,0.00,0.02,0.01,0.00,0.00,0.00,0.00,0.02,-0.02,0.00,0.00,-0.02,0.01,-0.00,-0.02,0.00,0.01,0.01,-0.01,0.01,-0.01,-0.01,-0.04,-0.01,-0.00,-0.01,0.00,0.03,0.00,0.00,-0.01,0.01,-0.01,-0.00,0.00,-0.01,0.01,-0.02,0.00,0.00,-0.01,-0.00,-0.01,0.00,-0.00,-0.01,0.00,0.00,-0.00,-0.01,-0.00,0.00,0.00,0.01,0.02,0.01,-0.01,0.01,0.00,-0.00,0.01,0.02,0.00,-0.01,0.00,0.02,0.01,0.01,0.01,-0.02,-0.01,-0.01,0.01,0.00,0.01,-0.01,-0.01,0.01,-0.00,-0.01,0.00
Integrated Gradient (x Input),0.01,-0.00,-0.01,0.00,-0.00,0.00,-0.00,0.01,-0.01,-0.00,-0.00,0.00,-0.01,-0.02,0.07,-0.00,0.01,-0.00,-0.02,0.01,0.00,-0.01,-0.01,-0.02,-0.04,-0.00,-0.01,-0.01,-0.01,-0.00,0.00,0.00,0.00,0.00,0.01,-0.00,0.01,0.00,0.01,-0.00,0.01,0.01,0.00,-0.01,-0.01,-0.01,-0.00,0.00,-0.01,0.02,-0.01,-0.01,0.01,0.00,-0.01,-0.01,0.01,0.01,0.01,-0.02,-0.01,0.07,-0.01,0.01,-0.02,-0.01,-0.01,0.00,-0.01,-

In [98]:
i = 3
exp = explanations_dict['explanations_11'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcdefgh Questi hanno tutta la mia stima perché hanno fatto il culo ai nazi rottinculo di Azov. Per i quali le nullità in servizio permanente come Gramellini trovavano le giustificazioni tipiche della merda che elargiscono con cura maniacale. Attaccati fortissimo al caxxo! Per l'eternità! [URL]


,@,us,##er,_,ab,##c,##de,##f,##gh,Questi,hanno,tutta,la,mia,stima,perché,hanno.1,fatto,il,culo,ai,naz,##i,ro,##ttin,##culo,di,Az,##ov,.,Per,i,quali,le,nu,##lli,##tà,in,servizio,permanente,come,Gra,##mel,##lini,trovavano,le.1,giustifica,##zioni,tipiche,della,merda,che,ela,##r,##gis,##cono,con,cura,mania,##cale,..1,Atta,##ccati,forti,##ssimo,al,ca,##xx,##o,!,Per.1,l,',et,##erni,##tà.1,!.1,[,UR,##L,]
Partition SHAP,0.03,-0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.05,-0.00,-0.01,-0.01,-0.06,-0.01,-0.02,-0.01,-0.00,0.00,-0.01,0.00,0.00,0.00,0.12,0.18,0.08,0.02,0.05,0.01,0.02,0.00,0.01,-0.00,-0.00,-0.00,-0.01,-0.00,-0.00,-0.01,-0.01,-0.01,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.01,-0.00,-0.05,-0.03,-0.01,-0.01,-0.00,-0.00,0.00,0.00,0.01,0.04,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.02,0.01
LIME,-0.01,-0.00,0.00,0.00,-0.00,-0.00,-0.01,0.00,0.00,0.03,0.00,-0.00,-0.00,-0.01,-0.01,0.01,-0.00,-0.01,0.01,-0.06,-0.00,-0.01,-0.01,0.07,0.18,0.11,0.03,0.00,-0.00,0.01,0.00,0.01,0.01,0.01,0.00,0.01,-0.00,-0.00,-0.00,-0.01,-0.00,0.00,0.01,0.01,-0.01,0.01,0.01,-0.01,-0.00,-0.00,0.03,-0.00,-0.02,-0.02,-0.01,-0.02,-0.01,-0.01,0.01,0.01,0.00,0.02,-0.01,-0.01,-0.02,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,-0.00,-0.00,0.00,-0.00,-0.00
Gradient,0.02,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.01,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.05,0.01,0.02,0.03,0.04,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.03,0.02,0.02,0.02,0.01,0.01,0.02,0.01,0.03,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.00,0.01,0.01,0.00,0.01,0.01,0.02,0.03,0.01,0.01,0.01
Gradient (x Input),-0.03,0.00,-0.01,-0.01,0.01,-0.00,-0.00,0.01,0.01,-0.01,0.00,0.00,-0.00,-0.01,-0.00,-0.00,-0.01,0.01,-0.01,-0.01,0.01,-0.00,0.00,0.06,0.03,0.05,-0.01,0.01,-0.01,0.00,0.01,-0.00,-0.01,0.01,0.01,0.00,0.01,-0.02,-0.04,0.01,0.01,0.00,-0.00,0.00,-0.00,0.01,0.02,0.01,0.01,0.00,0.03,-0.01,-0.01,0.00,-0.02,-0.02,-0.00,0.01,0.02,-0.02,0.02,0.03,-0.01,0.00,0.00,0.00,0.01,-0.02,-0.01,0.03,0.00,0.00,-0.03,-0.01,-0.01,0.00,0.04,-0.06,0.00,-0.00,0.02
Integrated Gradient,-0.04,-0.01,-0.00,-0.01,0.00,-0.01,-0.00,-0.01,0.01,0.00,0.00,-0.00,-0.01,0.00,-0.00,-0.01,-0.01,0.01,0.00,-0.01,0.01,0.03,-0.01,0.03,-0.03,-0.08,-0.01,-0.02,-0.01,-0.02,-0.01,-0.02,-0.00,-0.00,-0.01,-0.01,-0.01,-0.02,-0.02,0.00,-0.00,0.01,-0.01,0.00,-0.01,-0.01,-0.01,0.00,0.01,-0.01,-0.01,0.01,-0.02,0.03,-0.00,0.02,-0.03,0.00,-0.03,-0.03,-0.01,0.01,0.01,0.01,-0.01,-0.01,0.01,-0.02,0.01,-0.02,-0.00,-0.01,-0.00,0.00,-0.02,-0.00,-0.00,-0.02,-0.00,0.00,-0.02
Integrated Gradient (x Input),-0.01,-0.00,-0.01,-0.01,-0.00,-0.01,-0.00,0.00,0.00,-0.01,-0.00,-0.01,-0.01,-0.02,-0.02,0.03,-0.02,0.03,0.00,-0.02,-0.03,-0.06,-0.04,0.03,-0.00,0.10,0.03,0.01,-0.02,-0.01,0.01,0.00,-0.01,-0.01,0.00,-0.00,0.00,-0.02,-0.01,-0.00,0.01,0.00,0.00,0.00,-0.00,-0.01,0.01,0.00,-0.01,-0.01,0.04,-0.01,-0.01,-0.00,-0.01,-0.02,-0.01,-0.01,0.02,-0.01,-0.01,0.01,0.01,-0.01,-0.00,-0.01,-0.00,-0.01,-0.02,0.01,0.00,0.01,0.00,-0.00,-0.01,-0.00,0.02,-0.02,-0.01,-0.02,-0.01


In [99]:
i = 4
exp = explanations_dict['explanations_11'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcdefg Sai sparare ma sei rotto in culo


Token,@,us,##er,_,ab,##c,##de,##f,##g,Sai,sparare,ma,sei,rotto,in,culo
Partition SHAP,0.10,-0.02,-0.07,-0.08,-0.02,-0.05,0.03,-0.06,-0.02,-0.00,-0.17,0.00,0.04,0.10,0.01,0.21
LIME,0.01,-0.01,-0.03,-0.07,-0.00,-0.02,0.01,0.02,-0.01,0.07,-0.09,0.03,-0.04,0.33,0.08,0.20
Gradient,0.06,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.08,0.10,0.06,0.07,0.11,0.05,0.08
Gradient (x Input),0.03,-0.00,-0.02,-0.00,0.01,-0.03,0.02,0.02,-0.01,-0.06,0.01,-0.04,-0.01,0.14,0.05,-0.22
Integrated Gradient,0.16,0.08,0.01,0.01,-0.01,-0.01,-0.01,-0.09,-0.01,0.02,-0.01,-0.09,-0.13,-0.06,-0.06,-0.03
Integrated Gradient (x Input),0.10,-0.11,-0.06,-0.04,-0.00,-0.07,0.04,0.06,0.05,0.05,0.09,0.07,-0.07,0.15,0.02,0.03


In [100]:
i = 0
exp = explanations_dict['explanations_10'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abc @user_abcde cazzo ridi che hai una lesbica come punta


,@,us,##er,_,ab,##c,@.1,us.1,##er.1,_.1,ab.1,##c.1,##de,cazzo,ridi,che,hai,una,lesbica,come,punta
Partition SHAP,0.09,-0.08,-0.00,-0.13,0.04,-0.01,0.02,-0.09,-0.06,-0.03,0.00,-0.04,-0.00,0.04,-0.07,0.04,-0.10,-0.04,-0.04,-0.05,-0.02
LIME,-0.02,0.00,-0.01,-0.02,0.01,-0.03,0.11,0.05,0.03,0.02,-0.06,-0.04,-0.07,0.04,-0.05,0.04,-0.06,-0.09,-0.12,-0.04,-0.08
Gradient,0.04,0.02,0.02,0.02,0.01,0.02,0.04,0.02,0.02,0.03,0.02,0.03,0.05,0.10,0.11,0.08,0.05,0.04,0.09,0.03,0.05
Gradient (x Input),0.18,0.02,0.01,-0.02,0.00,-0.03,0.01,-0.01,-0.01,-0.02,0.00,-0.02,-0.04,-0.03,-0.06,-0.14,-0.03,-0.04,-0.27,0.00,-0.04
Integrated Gradient,0.15,-0.02,-0.05,-0.05,-0.01,0.00,-0.06,-0.02,-0.06,-0.03,0.04,-0.02,0.00,-0.01,0.01,0.05,-0.07,0.08,0.13,-0.00,0.08
Integrated Gradient (x Input),-0.00,-0.04,-0.03,-0.04,0.00,-0.02,-0.04,-0.04,-0.01,-0.05,-0.00,-0.03,0.00,0.03,0.02,-0.00,0.03,-0.13,-0.20,-0.23,0.05


In [101]:
i = 1
exp = explanations_dict['explanations_10'][i]
print(exp[i].text)
bench.show_table(exp)

È arrivato Kezman a Formello, dopo un summit di circa un paio di ore, ha detto che le piante stanno bene, bisogna da na romanella al muro de cinta, e poi ha detto che siete na banda de cazzari. Io mi fido…. #succhiacazzi #sieteilmaledellalazio


,È,arrivato,Ke,##z,##man,a,For,##mello,",",dopo,un,su,##mmi,##t,di,circa,un.1,paio,di.1,ore,",.1",ha,detto,che,le,piante,stanno,bene,",.2",bisogna,da,na,romane,##lla,al,muro,de,cinta,",.3",e,poi,ha.1,detto.1,che.1,siete,na.1,banda,de.1,ca,##zza,##ri,.,Io,mi,fido,[UNK],..1,#,succhia,##ca,##zzi,#.1,siete.1,##ilm,##ale,##della,##la,##zio
Partition SHAP,-0.00,0.08,-0.01,-0.00,-0.01,-0.02,0.00,0.01,-0.01,-0.01,0.05,-0.01,-0.02,-0.03,0.02,0.00,0.01,-0.01,0.01,0.03,0.01,-0.01,-0.05,-0.00,-0.01,-0.00,-0.00,-0.07,-0.01,-0.01,-0.01,-0.01,-0.01,-0.00,-0.00,0.00,0.01,0.03,0.00,-0.01,-0.02,-0.01,-0.05,0.00,0.00,-0.00,0.04,-0.00,-0.01,-0.02,-0.04,0.03,-0.02,-0.03,-0.02,-0.02,0.01,-0.01,-0.01,-0.00,-0.00,-0.02,-0.00,-0.01,-0.01,-0.01,-0.00,-0.00
LIME,0.01,0.04,0.00,0.00,-0.00,-0.00,-0.03,0.00,0.01,-0.01,0.02,0.01,-0.02,-0.02,0.00,-0.00,0.00,-0.01,0.01,0.00,0.00,0.01,-0.09,0.01,0.00,-0.01,0.04,-0.07,-0.00,-0.02,0.00,-0.01,-0.00,-0.01,-0.00,0.01,-0.02,0.01,0.00,-0.01,-0.04,0.02,-0.06,0.02,0.03,-0.02,0.07,0.03,-0.04,0.01,-0.00,0.01,0.00,-0.02,0.03,-0.01,0.00,0.01,-0.03,-0.01,0.01,0.00,-0.01,-0.00,-0.00,0.00,-0.00,0.01
Gradient,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.02,0.05,0.02,0.01,0.02,0.01,0.03,0.01,0.01,0.02,0.02,0.03,0.03,0.03,0.01,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01
Gradient (x Input),-0.02,0.05,0.02,0.01,-0.01,0.01,0.01,0.02,0.01,0.00,-0.01,-0.00,-0.00,-0.01,-0.01,-0.01,-0.00,-0.01,-0.01,0.01,0.00,0.02,-0.02,0.00,0.02,-0.02,0.01,0.00,-0.01,0.00,-0.00,0.03,0.01,0.00,-0.02,-0.02,0.01,0.00,-0.00,-0.01,-0.02,0.02,-0.00,0.01,-0.00,0.04,0.05,0.01,0.00,-0.01,0.01,-0.02,0.00,-0.01,-0.01,0.03,-0.03,0.08,0.04,0.01,0.01,0.06,-0.00,-0.01,-0.00,-0.01,-0.01,0.00
Integrated Gradient,0.01,-0.01,-0.03,0.02,-0.01,0.00,0.01,-0.01,0.03,0.02,0.01,-0.01,-0.01,0.01,-0.01,0.01,0.01,0.00,-0.00,-0.00,-0.00,-0.02,-0.01,-0.02,-0.02,0.02,0.01,-0.11,0.00,-0.01,-0.01,0.00,-0.01,0.00,-0.00,0.03,0.02,-0.01,-0.00,-0.01,-0.02,-0.02,0.01,0.01,-0.01,0.01,-0.03,0.05,0.03,0.00,0.01,0.01,0.00,-0.01,-0.05,-0.00,0.03,-0.04,-0.01,0.02,0.00,-0.00,-0.01,0.00,0.03,0.01,-0.00,0.03
Integrated Gradient (x Input),0.02,-0.02,-0.01,-0.00,0.01,-0.02,-0.01,0.01,0.00,-0.02,-0.00,-0.00,-0.00,-0.01,-0.00,-0.01,-0.00,-0.01,-0.00,-0.01,-0.00,-0.03,-0.12,-0.03,-0.03,0.00,-0.00,-0.09,-0.01,-0.02,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.01,0.00,-0.00,-0.00,-0.02,-0.01,-0.07,-0.04,-0.02,-0.00,0.04,-0.02,-0.01,-0.01,-0.01,-0.02,-0.02,-0.01,0.02,0.00,-0.01,-0.02,0.03,0.02,0.01,-0.01,0.01,-0.01,-0.02,-0.00,-0.00,0.01


In [102]:
i = 2
exp = explanations_dict['explanations_10'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcdefghij @user_abcde @user_abcde Complimenti per la cultura, la proprietà di linguaggio e il pelo sul petto!! Grande femminiello 🤗🤗🙉🙉


,@,us,##er,_,ab,##c,##de,##f,##ghi,##j,@.1,us.1,##er.1,_.1,ab.1,##c.1,##de.1,@.2,us.2,##er.2,_.2,ab.2,##c.2,##de.2,Complimenti,per,la,cultura,",",la.1,proprietà,di,linguaggio,e,il,pelo,sul,petto,!,!.1,Grande,femmini,##ello,[UNK]
Partition SHAP,0.03,-0.03,-0.01,-0.03,-0.01,-0.01,-0.00,-0.00,0.03,0.00,0.03,-0.03,-0.01,-0.02,0.01,0.00,-0.01,0.02,-0.02,-0.00,-0.01,0.01,-0.01,-0.01,-0.04,-0.01,-0.03,-0.02,0.02,-0.04,-0.01,0.00,-0.03,-0.00,-0.01,-0.02,-0.02,-0.02,-0.00,0.00,-0.05,-0.23,0.03,-0.06
LIME,0.01,-0.01,-0.01,-0.02,-0.01,-0.01,0.01,0.01,0.04,-0.00,0.01,0.00,-0.00,0.01,0.03,-0.00,0.01,0.02,-0.01,0.00,-0.00,0.01,-0.02,-0.00,-0.09,-0.04,-0.00,-0.03,0.01,0.02,0.03,0.03,-0.02,-0.01,0.00,-0.02,-0.04,-0.01,-0.00,-0.05,-0.13,0.04,0.11,-0.06
Gradient,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.01,0.01,0.02,0.03,0.01,0.02,0.02,0.02,0.02,0.02,0.03,0.02,0.03,0.03,0.05,0.05,0.18,0.06,0.03
Gradient (x Input),-0.01,-0.01,-0.01,0.00,0.00,0.00,0.01,-0.02,0.00,0.01,0.00,-0.01,-0.01,0.01,0.00,0.01,0.00,-0.03,-0.01,-0.01,-0.00,0.01,0.01,0.00,0.01,0.01,0.02,0.02,-0.11,0.01,-0.01,-0.02,-0.02,0.00,-0.00,0.03,-0.01,0.06,-0.09,-0.14,-0.03,0.02,-0.14,-0.04
Integrated Gradient,0.00,0.01,-0.01,0.03,-0.01,0.00,0.00,0.01,0.01,-0.00,0.01,0.02,0.01,0.00,-0.00,-0.01,-0.00,0.02,0.00,0.01,0.01,-0.03,-0.03,0.00,0.04,0.00,0.00,-0.03,0.01,-0.02,-0.04,-0.01,0.00,0.02,-0.01,-0.05,0.04,-0.03,-0.02,-0.05,-0.04,0.20,0.04,0.01
Integrated Gradient (x Input),-0.00,-0.01,-0.02,-0.01,-0.02,-0.02,-0.02,-0.02,0.02,0.01,-0.01,-0.01,-0.02,-0.01,-0.03,-0.01,-0.04,-0.00,-0.01,-0.04,-0.02,-0.03,-0.02,-0.03,-0.01,-0.03,-0.02,-0.06,-0.03,-0.04,-0.03,-0.04,-0.07,-0.01,0.01,-0.03,-0.02,-0.02,-0.05,-0.04,0.02,-0.02,0.02,0.01


In [103]:
i = 3
exp = explanations_dict['explanations_10'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcd Masturbanti a noi con quella pik da travione😅😂😂😂


Token,@,us,##er,_,ab,##c,##d,Mas,##turba,##nti,a,noi,con,quella,pi,##k,da,[UNK]
Partition SHAP,0.10,-0.05,-0.05,0.00,-0.02,-0.03,-0.04,-0.07,-0.23,-0.01,-0.01,0.01,-0.09,-0.10,-0.01,-0.01,-0.11,-0.06
LIME,0.01,-0.04,-0.09,-0.04,0.10,-0.10,0.01,-0.02,-0.13,-0.02,-0.03,-0.05,-0.11,-0.23,0.00,0.01,0.01,-0.01
Gradient,0.06,0.04,0.02,0.03,0.03,0.03,0.02,0.07,0.13,0.06,0.04,0.07,0.05,0.06,0.05,0.04,0.03,0.04
Gradient (x Input),-0.06,-0.03,-0.01,0.02,-0.05,0.00,-0.02,0.01,-0.22,-0.05,0.03,-0.03,-0.02,0.01,-0.14,-0.01,-0.07,0.01
Integrated Gradient,0.04,0.02,-0.06,-0.01,0.04,-0.06,-0.05,-0.06,0.10,0.05,0.10,0.08,0.01,0.12,0.09,0.05,0.04,0.01
Integrated Gradient (x Input),-0.02,-0.02,-0.05,-0.01,0.00,-0.02,-0.02,0.01,0.00,-0.02,-0.08,-0.23,0.04,-0.32,0.02,-0.12,-0.00,0.01


In [104]:
i = 4
exp = explanations_dict['explanations_10'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcdefgh CIAO DONOSO...DI KE PARLI??? 🤔IO NN O POSTATO NESSUN MEME VEDO SOLO UN RATTO KE FUMA UNA SIGARETTA.. SARÀ STATO KUEL CRETINO FEMMINIELLO DI MIO NIPOTE 🤬🤬🤬


,@,us,##er,_,ab,##c,##de,##f,##gh,CIA,##O,DON,##OS,##O.1,.,..1,..2,DI,K,##E,PAR,##LI,?,?.1,?.2,[UNK],N,##N,O,POS,##TA,##TO,NE,##SS,##UN,MEM,##E.1,V,##ED,##O.2,SOL,##O.3,UN,RA,##T,##TO.1,K.1,##E.2,FU,##MA,UNA,SI,##GA,##RE,##T.1,##TA.1,..3,..4,SA,##R,##À,STA,##TO.2,K.2,##UE,##L,CR,##ET,##INO,FE,##M,##MIN,##IE,##LL,##O.4,DI.1,MI,##O.5,N.1,##IP,##OT,##E.3,[UNK].1
Partition SHAP,0.04,-0.03,-0.01,-0.02,-0.02,-0.01,0.00,-0.01,0.01,-0.00,-0.03,-0.06,-0.00,-0.02,-0.01,0.01,0.01,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.01,-0.01,-0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.01,-0.00,-0.00,-0.00,-0.01,-0.00,-0.01,-0.01,-0.01,-0.01,-0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.01,-0.02,-0.00,-0.00,-0.01,-0.01,-0.02,0.00,0.00,-0.10,-0.03,-0.09,-0.05,-0.03,0.01,0.00,0.00,0.01,0.04,0.02,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.01,-0.00,-0.00,-0.02,-0.01,-0.07
LIME,-0.01,-0.01,0.01,-0.03,-0.00,-0.02,0.02,-0.00,-0.00,-0.00,-0.03,-0.00,-0.00,0.01,0.01,-0.00,0.01,0.02,0.02,-0.00,0.01,-0.01,-0.01,-0.03,0.00,-0.01,-0.01,-0.02,0.00,-0.03,0.02,-0.00,0.02,0.01,0.02,-0.00,0.01,-0.01,-0.02,0.01,-0.00,-0.01,0.04,0.02,0.01,-0.01,0.02,-0.02,0.00,-0.01,-0.04,-0.00,0.01,-0.03,-0.01,-0.01,0.01,0.02,-0.03,-0.01,-0.01,-0.02,-0.03,-0.02,0.00,0.01,-0.01,0.03,0.02,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.00,-0.01,-0.01,0.00,-0.00,-0.02,-0.02
Gradient,0.02,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.04,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.00,0.00,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.00,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.02,0.02,0.02,0.02,0.04,0.03,0.03,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
Gradient (x Input),0.04,-0.01,-0.01,0.02,-0.00,0.00,-0.00,0.00,0.01,0.01,-0.01,-0.01,-0.03,-0.02,0.00,0.01,0.00,0.00,-0.00,0.00,0.00,0.00,0.01,0.03,0.02,0.02,-0.01,-0.00,0.01,0.01,0.01,0.00,-0.00,-0.00,-0.01,0.01,-0.01,0.00,-0.01,0.00,-0.00,-0.01,0.04,-0.01,0.03,0.01,0.01,0.00,-0.01,0.01,-0.00,0.01,0.01,-0.00,0.00,0.01,-0.03,-0.04,0.00,0.01,0.10,-0.05,-0.02,-0.00,-0.00,-0.00,0.00,0.02,0.06,-0.00,-0.00,0.00,0.01,0.00,-0.01,0.01,-0.02,-0.00,0.01,-0.00,-0.00,-0.01,0.04
Integrated Gradient,-0.01,0.00,0.00,-0.01,-0.00,0.01,0.00,-0.00,0.03,-0.01,0.02,0.00,0.00,-0.01,0.00,-0.00,0.03,-0.01,0.01,-0.03,-0.02,0.00,-0.01,0.00,0.01,-0.02,-0.00,-0.00,0.01,-0.00,0.00,-0.01,0.01,0.00,0.01,0.00,0.01,0.01,0.02,0.04,0.01,0.01,0.01,-0.01,0.01,0.00,0.02,-0.01,0.02,-0.04,-0.06,-0.01,-0.02,-0.00,0.02,-0.01,0.01,0.00,-0.02,-0.01,0.00,-0.01,-0.02,0.01,-0.05,-0.01,0.01,0.01,-0.01,0.00,0.01,-0.00,0.00,0.06,0.03,0.01,0.01,0.02,-0.01,-0.01,0.01,0.01,-0.00
Integrated Gradient (x Input),0.01,-0.00,-0.01,-0.01,-0.01,-0.01,-0.01,-0.00,-0.01,-0.00,-0.00,0.01,0.00,0.01,-0.01,-0.01,-0.01,-0.00,-0.03,-0.02,-0.01,0.01,-0.02,-0.00,-0.00,0.00,0.01,0.00,0.01,-0.01,0.00,0.01,0.00,0.00,-0.00,-0.00,-0.00,-0.02,-0.02,0.01,-0.01,-0.01,0.00,0.02,-0.01,-0.00,-0.00,0.02,-0.00,0.01,-0.05,0.01,0.01,-0.00,-0.00,-0.00,-0.00,-0.02,-0.03,-0.02,-0.01,0.01,-0.00,-0.01,-0.01,0.03,-0.00,0.00,0.03,-0.01,-0.02,-0.06,-0.03,-0.02,0.04,0.04,-0.03,0.03,0.01,0.01,-0.02,-0.00,0.00


In [148]:
rationales_dict

{'@user_abcdefghi ma sei rotto in culo tu che cazzo significa': 'ma sei rotto in culo tu',
 '@user_abcdef Certo è un noto culattone @user_abcdefghij la carriera l‘ha fatta con Arpa testa bassa e culo alzato': 'Certo è un noto culattone @user_abcdefghijtesta bassa e culo alzato',
 '@user_abcd CHI ? AHHH QUESTO COGLIONE RICCHIONE SODOMITA PEDOFILO… 😂… HA ANCHE LA TESTA A FORMA DI GLANDE. STAI ATTENTO PELATO, CHE SIAMO IN TANTI A PISCIARTI IN TESTA, TI RICRESCONO I CAPELLI POI… NON ABBOCCATE SONO BALLE. IL PRIMO SCHIAFFO LO PRENDE LUI. 🎪🤡🤡🤡': 'QUESTO COGLIONE RICCHIONE SODOMITA PEDOFILO… 😂… HA ANCHE LA TESTA A FORMA DI GLANDEIL PRIMO SCHIAFFO LO PRENDE LUI. 🎪🤡🤡🤡',
 "@user_abcdefgh Questi hanno tutta la mia stima perché hanno fatto il culo ai nazi rottinculo di Azov. Per i quali le nullità in servizio permanente come Gramellini trovavano le giustificazioni tipiche della merda che elargiscono con cura maniacale. Attaccati fortissimo al caxxo! Per l'eternità! [URL]": 'ai nazi rottinculo di A